# TODO: Load weights_conv.h5 instead of weights_ft.h5

# Initializations

In [1]:
import os, shutil, importlib
import utils; importlib.reload(utils)

# os.environ["CUDA_VISIBLE_DEVICES"]=''

import numpy as np
import pandas as pd
import tensorflow as tf
import h5py
import keras

from glob import glob
from PIL import Image
from os.path import join as jp

from keras import layers, models, optimizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import image
from keras.utils import layer_utils

from sklearn import metrics

from IPython.display import FileLink


%matplotlib inline
np.set_printoptions(precision=4, linewidth=100)

Using TensorFlow backend.


Structure data

In [2]:
path = '../data/dogscats/'
utils.struct_dir(path, ['cat', 'dog'])

Directory structure already exists


Setup paths

In [3]:
path = '../data/dogscats/'
# path = '../data/dogscats/sample/'
path_model = '../data/dogscats/models/vgg16/'
path_test = path + 'test'
path_train = path + 'train'
path_valid = path + 'valid'
path_img_arrays = path + 'img_arrays/'

path_submit = path_model + 'submissions/'
path_checkpoint = path_model + 'checkpoints/'
path_results = path_model + 'results/'

# Load Model

- Load model and finetune weights

In [4]:
import vgg16; importlib.reload(vgg16)
from vgg16 import Vgg16

In [5]:
vgg = Vgg16()

Loading model weights


In [6]:
vgg.ft(2)
vgg.model.load_weights(path_model+'attempt3/checkpoints/weights_ft.h5') 

In [7]:
batch_size = 64

# Model Modifications

## Remove Dropout

- Split the model into conv and dense layers
- Precalculate the output of the convs layers
- Create new model with just the dense layers
- Train new model using output of conv layers as features

### Split Model

In [8]:
vgg.model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 224, 224, 3)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 226, 226, 3)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 224, 224, 64)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 226, 226, 64)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [10]:
layers_conv, layers_fc = utils.split_at(vgg.model, layers.Conv2D)

In [11]:
vgg_conv = models.Sequential(layers_conv)

### Load data and pre-calculate conv features

In [12]:
data_train = utils.load_array_bcolz(path_img_arrays+'train.bc')
data_valid = utils.load_array_bcolz(path_img_arrays+'valid.bc')
classes, filenames = utils.get_classes(path)
labels_train = classes[2]
labels_valid = classes[3]

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [11]:
# gen = image.ImageDataGenerator()
# batches_train = gen.flow(data_train, labels_train, batch_size=batch_size, shuffle=False)
# batches_valid = gen.flow(data_valid, labels_valid, batch_size=batch_size, shuffle=False)

In [12]:
# feats_conv_train = vgg_conv.predict_generator(batches_train, batches_train.n)
# feats_conv_valid = vgg_conv.predict_generator(batches_valid, batches_valid.n)

In [13]:
# utils.save_array_bcolz(path_img_arrays+'train_conv.b5', feats_conv_train)
# utils.save_array_bcolz(path_img_arrays+'valid_conv.b5', feats_conv_valid)

In [13]:
feats_conv_train = utils.load_array_bcolz(path_img_arrays+'train_conv.b5')
feats_conv_valid = utils.load_array_bcolz(path_img_arrays+'valid_conv.b5')

In [14]:
feats_conv_train.shape

(23000, 14, 14, 512)

### Create FC Model

Create fc model by creating a new model of the same architecture as the fc portion of VGG16 with dropout p set to 0. Then copy all the weights from the previous dense layers

In [34]:
vgg_fc = utils.create_model_from_layers(layers_fc)

In [35]:
utils.set_dropout(vgg_fc, 0)

In [36]:
opt = optimizers.Adam(lr=0.000001)
vgg_fc.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

### Train FC Model with conv features

In [37]:
callbacks = [ModelCheckpoint(
        path_checkpoint+'weights_fc.{epoch:02d}-{val_loss:.3f}-{val_acc:.3f}.h5',
        save_best_only=True, monitor='val_acc')]
vgg_fc.fit(feats_conv_train, labels_train, batch_size=batch_size, nb_epoch=1,
           validation_data=(feats_conv_valid, labels_valid), callbacks=callbacks)

Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 13s - loss: 0.1102 - acc: 0.9695 - val_loss: 0.1030 - val_acc: 0.9720

Remove non optimal saved weights

In [38]:
epoch_keep = 1
utils.rm_checkpoints(path_checkpoint+'weights_fc', epoch_keep)

### Model Checkpoint

In [58]:
vgg_fc.load_weights(glob(path_checkpoint+'weights_fc.*')[0])

### Create Submission

In [39]:
batches_test, probs_test = vgg.test(path_test, batch_size=batch_size)
np.save(path_results+'filenames_test_fc.npy', batches_test.filenames)
np.save(path_results+'probs_test_fc.npy', probs_test)

Found 12500 images belonging to 1 classes.


In [40]:
utils.create_submit(batches_test, probs_test[:, 1], clip=(0.025, 0.975),
                    fname=path_submit+'submit_fc.csv')

In [41]:
FileLink('../data/dogscats/models/vgg16/submissions/submit_fc.csv')

/home/ubuntu/projects/fast_ai/data/dogscats/models/vgg16/submissions/submit_fc.csv